In [1]:
pwd()

"/home/zyu20/git/BulkLMM.jl/test/notebooks"

In [2]:
cd("..")

In [3]:
Threads.nthreads()

24

In [4]:
include("BXDdata_for_test.jl"); ## Load BXD data used for testing:

In [5]:
using LoopVectorization

In [6]:
include("../src/scan_LiteQTL.jl"); ## Load multithreaded functions to be tested:

In [7]:
include("../src/parallel_helpers.jl");

In [24]:
include("testHelper.jl");

In [9]:
BLAS.set_num_threads(24)

In [10]:
BLAS.get_num_threads()

24

In [75]:
traitID = 1112

1112

In [76]:
@time results_scan_alt = scan_alt(reshape(pheno[:, traitID], :, 1), geno, kinship; reml = false);

  2.570421 seconds (2.54 M allocations: 1.168 GiB, 25.65% gc time)


In [92]:
pve_list = unique(round.(results_scan_alt[3]; digits = 3))

204-element Vector{Float64}:
 0.835
 0.861
 0.847
 0.828
 0.822
 0.84
 0.858
 0.849
 0.838
 0.834
 0.832
 0.826
 0.831
 ⋮
 0.741
 0.787
 0.765
 0.757
 0.768
 0.745
 0.75
 0.763
 0.767
 0.783
 0.939
 0.925

In [81]:
pve_list = collect(1:10) .* 0.1;

In [93]:
@time LODs_bulkscan = bulkscan(pheno, geno, kinship, pve_list)

285.272481 seconds (40.10 k allocations: 412.261 GiB, 10.58% gc time)


7321×35556 Matrix{Float64}:
 0.120594  0.130312  0.111546  0.0491128  …  0.0469524  0.354027   0.566444
 0.120594  0.130312  0.111546  0.0491128     0.0469524  0.354027   0.566444
 0.120594  0.130312  0.111546  0.0491128     0.0469524  0.354027   0.566444
 0.120594  0.130312  0.111546  0.0491128     0.0469524  0.354027   0.566444
 0.120594  0.130312  0.111546  0.0491128     0.0469524  0.354027   0.566444
 0.120594  0.130312  0.111546  0.0491128  …  0.0469524  0.354027   0.566444
 0.120594  0.130312  0.111546  0.0491128     0.0469524  0.354027   0.566444
 0.120594  0.130312  0.111546  0.0491128     0.0469524  0.354027   0.566444
 0.120594  0.130312  0.111546  0.0491128     0.0469524  0.354027   0.566444
 0.120594  0.130312  0.111546  0.0491128     0.0469524  0.354027   0.566444
 0.120594  0.130312  0.111546  0.0491128  …  0.0469524  0.354027   0.566444
 0.120598  0.130317  0.111549  0.0491136     0.046953   0.354036   0.566455
 0.241186  0.259606  0.198836  0.0611176     0.0621972  0.53

In [99]:
@time LODs_bulkscan = bulkscan(pheno, geno, kinship, [results_scan_alt[3][1]])

  1.740508 seconds (214 allocations: 2.050 GiB, 13.51% gc time)


7321×35556 Matrix{Float64}:
 0.0685133   0.0774162  0.0712343   …  0.0469135   0.126384    0.501837
 0.0685133   0.0774162  0.0712343      0.0469135   0.126384    0.501837
 0.0685133   0.0774162  0.0712343      0.0469135   0.126384    0.501837
 0.0685133   0.0774162  0.0712343      0.0469135   0.126384    0.501837
 0.0685133   0.0774162  0.0712343      0.0469135   0.126384    0.501837
 0.0685133   0.0774162  0.0712343   …  0.0469135   0.126384    0.501837
 0.0685133   0.0774162  0.0712343      0.0469135   0.126384    0.501837
 0.0685133   0.0774162  0.0712343      0.0469135   0.126384    0.501837
 0.0685133   0.0774162  0.0712343      0.0469135   0.126384    0.501837
 0.0685133   0.0774162  0.0712343      0.0469135   0.126384    0.501837
 0.0685133   0.0774162  0.0712343   …  0.0469135   0.126384    0.501837
 0.068516    0.0774187  0.0712363      0.0469141   0.126386    0.501847
 0.156148    0.147156   0.12592        0.0563397   0.1673      0.706861
 ⋮                                  

In [100]:
hcat(results_scan_alt[4], LODs_bulkscan[:, traitID])

7321×2 Matrix{Float64}:
 0.381039     0.382287
 0.381039     0.382287
 0.381039     0.382287
 0.381039     0.382287
 0.381039     0.382287
 0.381039     0.382287
 0.381039     0.382287
 0.381039     0.382287
 0.381039     0.382287
 0.381039     0.382287
 0.381039     0.382287
 0.381047     0.382295
 0.576593     0.570371
 ⋮            
 0.000186308  0.000594774
 0.000186308  0.000594774
 0.000186308  0.000594774
 0.0127896    0.0101812
 0.0869155    0.0711951
 0.0869155    0.0711951
 0.428963     0.365588
 0.435515     0.369203
 0.435687     0.369459
 0.425038     0.360271
 0.772675     0.629874
 0.772675     0.629874

In [101]:
sumSqDiff(reshape(results_scan_alt[4], :, 1), reshape(LODs_bulkscan[:, traitID], :, 1))

31.80946174563183